# Katherine's Notebook

In [1]:
import pandas as pd                     
import matplotlib.pyplot as plt      
import numpy as np        
import pandas
import matplotlib as plt
from os import listdir
from os.path import isfile, join
mypath="../resources/raw/Aqi_cbsa"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [5]:
print(len(onlyfiles))
onlyfiles

41


['annual_aqi_by_cbsa_1980.csv',
 'annual_aqi_by_cbsa_1981.csv',
 'annual_aqi_by_cbsa_1982.csv',
 'annual_aqi_by_cbsa_1983.csv',
 'annual_aqi_by_cbsa_1984.csv',
 'annual_aqi_by_cbsa_1985.csv',
 'annual_aqi_by_cbsa_1986.csv',
 'annual_aqi_by_cbsa_1987.csv',
 'annual_aqi_by_cbsa_1988.csv',
 'annual_aqi_by_cbsa_1989.csv',
 'annual_aqi_by_cbsa_1990.csv',
 'annual_aqi_by_cbsa_1991.csv',
 'annual_aqi_by_cbsa_1992.csv',
 'annual_aqi_by_cbsa_1993.csv',
 'annual_aqi_by_cbsa_1994.csv',
 'annual_aqi_by_cbsa_1995.csv',
 'annual_aqi_by_cbsa_1996.csv',
 'annual_aqi_by_cbsa_1997.csv',
 'annual_aqi_by_cbsa_1998.csv',
 'annual_aqi_by_cbsa_1999.csv',
 'annual_aqi_by_cbsa_2000.csv',
 'annual_aqi_by_cbsa_2001.csv',
 'annual_aqi_by_cbsa_2002.csv',
 'annual_aqi_by_cbsa_2003.csv',
 'annual_aqi_by_cbsa_2004.csv',
 'annual_aqi_by_cbsa_2005.csv',
 'annual_aqi_by_cbsa_2006.csv',
 'annual_aqi_by_cbsa_2007.csv',
 'annual_aqi_by_cbsa_2008.csv',
 'annual_aqi_by_cbsa_2009.csv',
 'annual_aqi_by_cbsa_2010.csv',
 'annual

In [3]:
#create dataframe of list of files
dfs=[]
for file in onlyfiles: 
    try:
        dfs.append(pd.read_csv(mypath+"/"+file))
    except: 
        print(file)
len(dfs)

41

In [6]:
# name dfs[0]
df_initial=dfs[0]
merge_df=df_initial
merge_df

,CBSA,CBSA Code,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10
0,"Akron, OH",10420,1980,366,17,119,171,57,2,0,205,164,109,9,5,55,297,0,0
1,"Albany-Schenectady-Troy, NY",10580,1980,365,95,183,72,15,0,0,200,122,69,1,0,90,274,0,0
2,"Albuquerque, NM",10740,1980,366,160,155,30,12,9,0,257,118,54,126,48,192,0,0,0
3,"Alexandria, LA",10780,1980,183,152,25,5,1,0,0,169,71,35,0,0,183,0,0,0
4,"Allentown-Bethlehem-Easton, PA-NJ",10900,1980,365,104,157,58,36,9,1,357,159,69,14,91,124,136,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,"Yakima, WA",49420,1980,280,246,32,2,0,0,0,108,52,28,280,0,0,0,0,0
319,"York-Hanover, PA",49620,1980,358,139,139,51,24,5,0,208,147,60,1,92,117,148,0,0
320,"Youngstown-Warren-Boardman, OH-PA",49660,1980,366,62,184,108,11,1,0,218,119,84,6,205,62,93,0,0
321,"Yuba City, CA",49700,1980,45,45,0,0,0,0,0,42,30,21,0,0,45,0,0,0


In [9]:
#
print(merge_df.shape)
for df in dfs[1:]: 
    merge_df = pd.concat([merge_df, df])
print(merge_df.shape)

(323, 19)
(19823, 19)


In [10]:
merge_df

,CBSA,CBSA Code,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10
0,"Akron, OH",10420,1980,366,17,119,171,57,2,0,205,164,109,9,5,55,297,0,0
1,"Albany-Schenectady-Troy, NY",10580,1980,365,95,183,72,15,0,0,200,122,69,1,0,90,274,0,0
2,"Albuquerque, NM",10740,1980,366,160,155,30,12,9,0,257,118,54,126,48,192,0,0,0
3,"Alexandria, LA",10780,1980,183,152,25,5,1,0,0,169,71,35,0,0,183,0,0,0
4,"Allentown-Bethlehem-Easton, PA-NJ",10900,1980,365,104,157,58,36,9,1,357,159,69,14,91,124,136,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,"Worcester, MA-CT",49340,2020,61,48,13,0,0,0,0,63,55,38,1,3,25,0,32,0
336,"Yakima, WA",49420,2020,31,18,13,0,0,0,0,90,75,46,0,0,0,0,30,1
337,"Youngstown-Warren-Boardman, OH-PA",49660,2020,60,55,5,0,0,0,0,67,50,34,0,0,35,0,25,0
338,"Yuba City, CA",49700,2020,52,31,21,0,0,0,0,95,82,45,0,0,0,0,52,0


In [13]:
#number of rows for each year
merge_df["Year"].value_counts().sort_index()

1980    323
1981    328
1982    319
1983    327
1984    309
1985    348
1986    368
1987    387
1988    399
1989    440
1990    473
1991    487
1992    499
1993    499
1994    503
1995    521
1996    519
1997    518
1998    503
1999    539
2000    564
2001    568
2002    562
2003    568
2004    558
2005    556
2006    544
2007    540
2008    536
2009    543
2010    539
2011    543
2012    532
2013    531
2014    528
2015    532
2016    533
2017    536
2018    532
2019    529
2020    340
Name: Year, dtype: int64